In [74]:
#Import needed packages
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
import folium 

## Data Preparation

### Preprocessing

In [3]:
#Download the data and change into df
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Data downloaded!


### Filtering Staten Island

In [4]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [5]:
SI = neighborhoods[neighborhoods['Borough'] == 'Staten Island']
print(SI.head())

           Borough   Neighborhood   Latitude  Longitude
197  Staten Island     St. George  40.644982 -74.079353
198  Staten Island   New Brighton  40.640615 -74.087017
199  Staten Island      Stapleton  40.626928 -74.077902
200  Staten Island       Rosebank  40.615305 -74.069805
201  Staten Island  West Brighton  40.631879 -74.107182


### Data Visualization - Map

In [6]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of Manhattan using latitude and longitude values
map_SI = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(SI['Latitude'], SI['Longitude'], SI['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SI)  
    
map_SI

### Choose Richmond County and around

In [7]:
print(SI['Neighborhood'].unique())
print(SI['Neighborhood'].count())

['St. George' 'New Brighton' 'Stapleton' 'Rosebank' 'West Brighton'
 'Grymes Hill' 'Todt Hill' 'South Beach' 'Port Richmond'
 "Mariner's Harbor" 'Port Ivory' 'Castleton Corners' 'New Springville'
 'Travis' 'New Dorp' 'Oakwood' 'Great Kills' 'Eltingville' 'Annadale'
 'Woodrow' 'Tottenville' 'Tompkinsville' 'Silver Lake' 'Sunnyside'
 'Park Hill' 'Westerleigh' 'Graniteville' 'Arlington' 'Arrochar'
 'Grasmere' 'Old Town' 'Dongan Hills' 'Midland Beach' 'Grant City'
 'New Dorp Beach' 'Bay Terrace' 'Huguenot' 'Pleasant Plains'
 'Butler Manor' 'Charleston' 'Rossville' 'Arden Heights' 'Greenridge'
 'Heartland Village' 'Chelsea' 'Bloomfield' 'Bulls Head' 'Richmond Town'
 'Shore Acres' 'Clifton' 'Concord' 'Emerson Hill' 'Randall Manor'
 'Howland Hook' 'Elm Park' 'Manor Heights' 'Willowbrook' 'Sandy Ground'
 'Egbertville' "Prince's Bay" 'Lighthouse Hill' 'Richmond Valley'
 'Fox Hills']
63


In [8]:
SI_richmond = SI[SI['Neighborhood'].str.contains('Richmond')].reset_index(drop=True)
SI_richmond

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,Port Richmond,40.633669,-74.129434
1,Staten Island,Richmond Town,40.569606,-74.134057
2,Staten Island,Richmond Valley,40.519541,-74.229571


## Call for API

In [9]:
CLIENT_ID = 'WNZA2GX2GEXFJJLO4O12OMVC21HQP53ONV0E5RC0XSVEB5MX' # your Foursquare ID
CLIENT_SECRET = 'D5NIICUAOVHBXLNOHJXMNECQMENS12UUFFVRI5EBZAZQMMZ0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Get Venue Catogory

In [10]:
neighborhood_latitude = SI_richmond.loc[0, 'Latitude']
neighborhood_longitude = SI_richmond.loc[0, 'Longitude']
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WNZA2GX2GEXFJJLO4O12OMVC21HQP53ONV0E5RC0XSVEB5MX&client_secret=D5NIICUAOVHBXLNOHJXMNECQMENS12UUFFVRI5EBZAZQMMZ0&v=20180605&ll=40.63366930554365,-74.12943426797008&radius=1000&limit=100'

In [11]:
results = requests.get(url).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,CakeChef's Cookie Jar,Dessert Shop,40.626323,-74.130702
1,Egger's Ice Cream Parlor,Ice Cream Shop,40.626235,-74.129716
2,Ralph's Italian Ices,Ice Cream Shop,40.630415,-74.139646
3,Call It A Wrap,New American Restaurant,40.626147,-74.129899
4,Denino's Pizzeria Tavern,Pizza Place,40.630174,-74.140226


### Statistics of Richmond County and around

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue ID',
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
richmond_venues = getNearbyVenues(names=SI_richmond['Neighborhood'],
                                   latitudes=SI_richmond['Latitude'],
                                   longitudes=SI_richmond['Longitude']
                                  )

Port Richmond
Richmond Town
Richmond Valley


In [16]:
richmond_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
Neighborhood,,,,,,,
Port Richmond,5,5,5,5,5,5,5
Richmond Town,5,5,5,5,5,5,5
Richmond Valley,10,10,10,10,10,10,10


In [17]:
richmond_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,Port Richmond,40.633669,-74.129434,Enterprise Rent-A-Car,40.636864,-74.127079,4c446c9add1f2d7ff5ca7ff9,Rental Car Location
1,Port Richmond,40.633669,-74.129434,Dunkin' Donuts,40.637225,-74.127623,4baf712cf964a52020003ce3,Donut Shop
2,Port Richmond,40.633669,-74.129434,Shaolin Kung Fu Temple,40.631349,-74.131964,4ca5d8a4a643b713bacde6a6,Martial Arts Dojo
3,Port Richmond,40.633669,-74.129434,Val's Pizza,40.637111,-74.127500,4c6969f13bad2d7f46c0afee,Pizza Place
4,Port Richmond,40.633669,-74.129434,Top Notch Security,40.637409,-74.129852,530e105211d2b322d4cc391e,Business Service
5,Richmond Town,40.569606,-74.134057,Historic Richmond Town,40.572803,-74.133430,4bdc7a0803be20a1c3a6849b,History Museum
6,Richmond Town,40.569606,-74.134057,Bella Nail Spa,40.573094,-74.136667,4bf969a95ec320a1922a8ad3,Spa
7,Richmond Town,40.569606,-74.134057,Casa verde,40.573251,-74.136378,5079ccdce4b065f1dd8155ad,Italian Restaurant
8,Richmond Town,40.569606,-74.134057,The Bagel Depot,40.572800,-74.133424,4bdc1f0e63c5c9b68a742a68,Bagel Shop
9,Richmond Town,40.569606,-74.134057,Moonstruck Cafe,40.573073,-74.136913,50b7cd06e4b0dd3bd93d0eb7,Café


In [18]:
# one hot encoding
richmond_onehot = pd.get_dummies(richmond_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
richmond_onehot['Neighborhood'] = richmond_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [richmond_onehot.columns[-1]] + list(richmond_onehot.columns[:-1])
richmond_onehot = richmond_onehot[fixed_columns]

richmond_onehot.head()

,Neighborhood,Bagel Shop,Bank,Bubble Tea Shop,Business Service,Café,Construction & Landscaping,Deli / Bodega,Donut Shop,Fast Food Restaurant,Food,History Museum,Italian Restaurant,Martial Arts Dojo,Mexican Restaurant,Pizza Place,Rental Car Location,Sandwich Place,Smoothie Shop,Spa,Train Station
0,Port Richmond,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,Port Richmond,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Port Richmond,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Port Richmond,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Port Richmond,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
richmond_grouped = richmond_onehot.groupby('Neighborhood').mean().reset_index()
richmond_grouped

,Neighborhood,Bagel Shop,Bank,Bubble Tea Shop,Business Service,Café,Construction & Landscaping,Deli / Bodega,Donut Shop,Fast Food Restaurant,Food,History Museum,Italian Restaurant,Martial Arts Dojo,Mexican Restaurant,Pizza Place,Rental Car Location,Sandwich Place,Smoothie Shop,Spa,Train Station
0,Port Richmond,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.2,0.0,0.0,0.0,0.0
1,Richmond Town,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
2,Richmond Valley,0.0,0.1,0.1,0.0,0.0,0.1,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.1,0.0,0.1


In [20]:
num_top_venues = 5

for hood in richmond_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = richmond_grouped[richmond_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Port Richmond----
                 venue  freq
0     Business Service   0.2
1  Rental Car Location   0.2
2           Donut Shop   0.2
3          Pizza Place   0.2
4    Martial Arts Dojo   0.2


----Richmond Town----
                venue  freq
0          Bagel Shop   0.2
1                Café   0.2
2                 Spa   0.2
3  Italian Restaurant   0.2
4      History Museum   0.2


----Richmond Valley----
                        venue  freq
0               Train Station   0.1
1          Mexican Restaurant   0.1
2             Bubble Tea Shop   0.1
3               Smoothie Shop   0.1
4  Construction & Landscaping   0.1




In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = richmond_grouped['Neighborhood']

for ind in np.arange(richmond_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(richmond_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Port Richmond,Donut Shop,Rental Car Location,Pizza Place,Business Service,Martial Arts Dojo,Train Station,Bank,Bubble Tea Shop,Café,Construction & Landscaping
1,Richmond Town,Bagel Shop,Café,Italian Restaurant,History Museum,Spa,Donut Shop,Bank,Bubble Tea Shop,Business Service,Construction & Landscaping
2,Richmond Valley,Train Station,Fast Food Restaurant,Bank,Bubble Tea Shop,Construction & Landscaping,Deli / Bodega,Food,Mexican Restaurant,Sandwich Place,Smoothie Shop


### K-Means Clustering Method

In [23]:
# set number of clusters
kclusters = 3

richmond_grouped_clustering = richmond_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(richmond_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1], dtype=int32)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
richmond_merged = SI_richmond

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
richmond_merged = richmond_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

richmond_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,Port Richmond,40.633669,-74.129434,0,Donut Shop,Rental Car Location,Pizza Place,Business Service,Martial Arts Dojo,Train Station,Bank,Bubble Tea Shop,Café,Construction & Landscaping
1,Staten Island,Richmond Town,40.569606,-74.134057,2,Bagel Shop,Café,Italian Restaurant,History Museum,Spa,Donut Shop,Bank,Bubble Tea Shop,Business Service,Construction & Landscaping
2,Staten Island,Richmond Valley,40.519541,-74.229571,1,Train Station,Fast Food Restaurant,Bank,Bubble Tea Shop,Construction & Landscaping,Deli / Bodega,Food,Mexican Restaurant,Sandwich Place,Smoothie Shop


In [31]:
richmond_merged.loc[richmond_merged['Cluster Labels'] == 0, richmond_merged.columns[[1] + list(range(5, richmond_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Port Richmond,Donut Shop,Rental Car Location,Pizza Place,Business Service,Martial Arts Dojo,Train Station,Bank,Bubble Tea Shop,Café,Construction & Landscaping


In [32]:
richmond_merged.loc[richmond_merged['Cluster Labels'] == 1, richmond_merged.columns[[1] + list(range(5, richmond_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Richmond Valley,Train Station,Fast Food Restaurant,Bank,Bubble Tea Shop,Construction & Landscaping,Deli / Bodega,Food,Mexican Restaurant,Sandwich Place,Smoothie Shop


In [33]:
richmond_merged.loc[richmond_merged['Cluster Labels'] == 2, richmond_merged.columns[[1] + list(range(5, richmond_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Richmond Town,Bagel Shop,Café,Italian Restaurant,History Museum,Spa,Donut Shop,Bank,Bubble Tea Shop,Business Service,Construction & Landscaping


## Menu for Each Venue and likes

In [34]:
richmond_menu = richmond_venues[richmond_venues['Venue Category'].str.contains('Pizza Place|Fast Food Restaurant|Mexican Restaurant|Italian Restaurant|Sandwich Place')].reset_index(drop=True)
richmond_menu

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,Port Richmond,40.633669,-74.129434,Val's Pizza,40.637111,-74.127500,4c6969f13bad2d7f46c0afee,Pizza Place
1,Richmond Town,40.569606,-74.134057,Casa verde,40.573251,-74.136378,5079ccdce4b065f1dd8155ad,Italian Restaurant
2,Richmond Valley,40.519541,-74.229571,Qdoba Mexican Grill,40.516974,-74.234161,52aa599011d250326f249f53,Mexican Restaurant
3,Richmond Valley,40.519541,-74.229571,SUBWAY,40.516739,-74.233441,4be9ab1161aca59345d58200,Sandwich Place
4,Richmond Valley,40.519541,-74.229571,McDonald's,40.517720,-74.234751,4c5a2399ec2520a16bfc5012,Fast Food Restaurant


In [234]:
CLIENT_ID = 'WNZA2GX2GEXFJJLO4O12OMVC21HQP53ONV0E5RC0XSVEB5MX' # your Foursquare ID
CLIENT_SECRET = 'D5NIICUAOVHBXLNOHJXMNECQMENS12UUFFVRI5EBZAZQMMZ0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [38]:
VENUE_ID = richmond_menu['Venue ID'].loc[0]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results = requests.get(url).json()

In [39]:
results

{'meta': {'code': 200, 'requestId': '5c543c77351e3d1dfbfdbc08'},
 'response': {'menu': {'menus': {'count': 0}}}}

In [42]:
VENUE_ID = richmond_menu['Venue ID'].loc[1]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results = requests.get(url).json()

In [43]:
results

{'meta': {'code': 200, 'requestId': '5c543e9b351e3d1df75398c1'},
 'response': {'menu': {'menus': {'count': 0}}}}

In [180]:
VENUE_ID = richmond_menu['Venue ID'].loc[2]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results = requests.get(url).json()['response']['menu']['menus']['items']

In [181]:
results[0]['entries']['count']

20

In [182]:
totalfinal = []

totalfinal.append([(
    results[0]['entries']['items'][i]['name'].split(':')[0], 
    results[0]['entries']['items'][i]['name'].split(':')[1]) for i in range(0,20)])

totalf = pd.DataFrame([item for Total_ in totalfinal for item in Total_])
totalf.columns = ['Name','Content']

In [183]:
totalf

,Name,Content
0,Fan Favorites,Flavor Favoritos! Craveable Fan Favorites Tha...
1,New & Lto Items,Come in Today to Try Our New Exciting Menu It...
2,"Lunchtime, Dinnertime & Anytime",So Many Delicious Different Flavor Combinatio...
3,Quesos,Crave-Worthy Quesos So Packed with Flavor You...
4,To Share,"Bring Your Family, Co-Worker and Best Amigo t..."
5,Customizable Entrees,"Build Your Own Mexican Dish - Nachos, Tacos, ..."
6,Signature Eats,Chef-Inspired Mashups That Will Change the Wa...
7,Vegetarian Eats,"Vegetarians Welcome! Our Fresh Cut Produce, A..."
8,Kids Meal,a Mini Mexican Feast for Your Little Ones (Ni...
9,Chips & Dips,What Goes Better with Chips Than Dips?!? Dunk...


In [169]:
VENUE_ID = richmond_menu['Venue ID'].loc[2]
url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results_like = requests.get(url).json()['response']['likes']['count']
results_like

4

In [46]:
VENUE_ID = richmond_menu['Venue ID'].loc[3]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results = requests.get(url).json()

In [47]:
results

{'meta': {'code': 200, 'requestId': '5c543ede1ed2193b439b8ac4'},
 'response': {'menu': {'menus': {'count': 0}}}}

In [184]:
VENUE_ID = richmond_menu['Venue ID'].loc[4]
url = 'https://api.foursquare.com/v2/venues/{}/menu?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results4 = requests.get(url).json()['response']['menu']['menus']['items']

In [178]:
results4[0]['entries']['count']

9

In [203]:
totalfinal2 = []
for i in range(0,9):
    if i == 7:
        totalfinal2.append([(results4[0]['entries']['items'][7]['name'].split(':')[0],
                            'None')])
    else:
        totalfinal2.append([(results4[0]['entries']['items'][i]['name'].split(':')[0], 
                             results4[0]['entries']['items'][i]['name'].split(':')[1])])

totalf2 = pd.DataFrame([item for Total_ in totalfinal2 for item in Total_])
totalf2.columns = ['Name','Content']

In [204]:
totalf2

,Name,Content
0,Burgers,"When It Comes to Our Burgers, We Mean Busines..."
1,Chicken & Sandwiches,Irresistible Flavors Come to the Table. Enjoy...
2,Breakfast,Meals Include Hash Browns and Small Coffee Or...
3,Snacks & Sides,Grab Somethin’ for Your Snacks & Sides. We Kn...
4,Desserts & Shakes,Our Cool Options Don’t Disappoint.
5,Happy Meal,You Want the Best for Your Family. So Do We. ...
6,Salads,Get Your Greens with Our Freshly-Prepared Sal...
7,Mccafé®,None
8,Beverages,Hit Refresh When You Sip Our Drinks. Satisfy ...


In [205]:
VENUE_ID = richmond_menu['Venue ID'].loc[4]
url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
results_like2 = requests.get(url).json()['response']['likes']['count']
results_like2

9